<a href="https://colab.research.google.com/github/UiinKim/UiinKim/blob/main/Word_Level_Neural_Machine_Translation_Tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.
Found GPU at: /device:GPU:0


In [2]:
import os
import re
import shutil
import zipfile

import numpy as np
import pandas as pd
import tensorflow as tf
import unicodedata
import urllib3
from tensorflow.keras.layers import Embedding, Dense,GRU
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer

In [3]:
from google.colab import files
uploaded=files.upload()

Saving fra.txt to fra.txt


In [4]:
num_samples=33000

In [5]:
def to_ascii(s):
  #프랑스어의 악센트 삭제
  return ''.join(c for c in unicodedata.normalize('NFD',s) if unicodedata.category(c)!='Mn')

def preprocess_sentence(sent):
  sent=to_ascii(sent.lower())

  #단어와 구두점 사이에 공백 추가
  #i am a student. -> i am a student .
  sent = re.sub(r"([?.!,¿])", r" \1", sent)

  #(a-z, A-Z, ".", "?", "!", ",")을 제외하고 모두 공백으로 변환
  sent=re.sub(r"[^a-zA-Z!.?]+", " ", sent)

  #다수 개의 공백을 하나의 공백으로 치환
  sent=re.sub(r"\s+", " ", sent)
  return sent

In [6]:
#전처리 테스트
eng_sent=u"Have you had dinner?"
fra_sent=u"Avez-vous déjà diné?"

print("전처리 전 영어 문장 : ", eng_sent)
print("전처리 후 영어 문장 : ", preprocess_sentence(eng_sent))
print("전처리 전 프랑스어 문장 : ", fra_sent)
print("전처리 후 프랑스어 문장 : ", preprocess_sentence(fra_sent))

전처리 전 영어 문장 :  Have you had dinner?
전처리 후 영어 문장 :  have you had dinner ?
전처리 전 프랑스어 문장 :  Avez-vous déjà diné?
전처리 후 프랑스어 문장 :  avez vous deja dine ?


In [7]:
x=pd.read_table('fra.txt')
x.head()

,Go.,Va !,CC-BY 2.0 (France) Attribution: tatoeba.org #2877272 (CM) & #1158250 (Wittydev)
0,Go.,Marche.,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
1,Go.,En route !,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
2,Go.,Bouge !,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
3,Hi.,Salut !,CC-BY 2.0 (France) Attribution: tatoeba.org #5...
4,Hi.,Salut.,CC-BY 2.0 (France) Attribution: tatoeba.org #5...


In [8]:
def load_preprocessed_data():
  encoder_input, decoder_input, decoder_target=[],[],[]

  with open("fra.txt", "r") as lines:
    for i, line in enumerate(lines):
      src_line, tar_line, _=line.strip().split('\t')

      src_line=[w for w in preprocess_sentence(src_line).split()]

      tar_line=preprocess_sentence(tar_line)
      tar_line_in=[w for w in ("<sos> " + tar_line).split()]
      tar_line_out=[w for w in (tar_line + " <eos>").split()]

      encoder_input.append(src_line)
      decoder_input.append(tar_line_in)
      decoder_target.append(tar_line_out)

      if i==num_samples -1:
        break
        #최대치 넘어가면 멈추기
  return encoder_input, decoder_input, decoder_target


In [9]:
sents_en_in, sents_fra_in, sents_fra_out=load_preprocessed_data()
print("인코더의 입력 : ", sents_en_in[:5])
print('디코더의 입력 : ', sents_fra_in[:5])
print("디코더의 레이블 : ", sents_fra_out[:5])

인코더의 입력 :  [['go', '.'], ['go', '.'], ['go', '.'], ['go', '.'], ['hi', '.']]
디코더의 입력 :  [['<sos>', 'va', '!'], ['<sos>', 'marche', '.'], ['<sos>', 'en', 'route', '!'], ['<sos>', 'bouge', '!'], ['<sos>', 'salut', '!']]
디코더의 레이블 :  [['va', '!', '<eos>'], ['marche', '.', '<eos>'], ['en', 'route', '!', '<eos>'], ['bouge', '!', '<eos>'], ['salut', '!', '<eos>']]


In [10]:
tokenizer_en=Tokenizer(filters="",lower=False)
tokenizer_en.fit_on_texts(sents_en_in)
encoder_input=tokenizer_en.texts_to_sequences(sents_en_in)
encoder_input=pad_sequences(encoder_input, padding='post')

tokenizer_fra=Tokenizer(filters="",lower=False)
tokenizer_fra.fit_on_texts(sents_fra_in)
tokenizer_fra.fit_on_texts(sents_fra_out)
decoder_input=tokenizer_fra.texts_to_sequences(sents_fra_in)
decoder_input=pad_sequences(decoder_input, padding="post")
decoder_target=tokenizer_fra.texts_to_sequences(sents_fra_out)
decoder_target=pad_sequences(decoder_target, padding='post')

In [11]:
print('인코더의 입력의 크기 : ', encoder_input.shape)
print("디코더의 입력의 크기 : ", decoder_input.shape)
print("디코더의 레이블의 크기 : ", decoder_target.shape)

인코더의 입력의 크기 :  (33000, 7)
디코더의 입력의 크기 :  (33000, 16)
디코더의 레이블의 크기 :  (33000, 16)


In [12]:
src_vocab_size=len(tokenizer_en.word_index)+1
tar_vocab_size=len(tokenizer_fra.word_index)+1
print("영어 집합의 크기 : ", src_vocab_size)
print("프랑스어 집합의 크기 : ", tar_vocab_size)

영어 집합의 크기 :  4481
프랑스어 집합의 크기 :  7873


In [13]:
src_to_index=tokenizer_en.word_index
index_to_src=tokenizer_en.index_word
tar_to_index=tokenizer_fra.word_index
index_to_tar=tokenizer_fra.index_word

In [16]:
#테스트 데이터를 분리하기 위해 랜덤으로 섞어준다.
indices=np.arange(encoder_input.shape[0]) #정수만 구하기 때문에shape[0]
np.random.shuffle(indices)
print('랜덤 시퀀스 : ', indices)

랜덤 시퀀스 :  [11770 19118  8046 ... 24996 32818 25037]


In [17]:
encoder_input=encoder_input[indices]
decoder_input=decoder_input[indices]
decoder_target=decoder_target[indices]

In [18]:
encoder_input[30997]

array([  2, 556,   3,  45,   1,   0,   0], dtype=int32)

In [19]:
decoder_input[30997]

array([   2,    4,    9,   16, 2290,    1,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0], dtype=int32)

In [20]:
decoder_target[30997]

array([   4,    9,   16, 2290,    1,    3,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0], dtype=int32)

In [21]:
#검증데이터
n_of_val=int(33000*0.1)
print("검증데이터의 수 : ", n_of_val)

검증데이터의 수 :  3300


In [22]:
encoder_input_train=encoder_input[:-n_of_val]
decoder_input_train=decoder_input[:-n_of_val]
decoder_target_train=decoder_target[:-n_of_val]

encoder_input_test=encoder_input[-n_of_val:]
decoder_input_test=decoder_input[-n_of_val:]
decoder_target_test=decoder_target[-n_of_val:]

In [23]:
print('훈련 source 데이터의 크기 :',encoder_input_train.shape)
print('훈련 target 데이터의 크기 :',decoder_input_train.shape)
print('훈련 target 레이블의 크기 :',decoder_target_train.shape)
print('테스트 source 데이터의 크기 :',encoder_input_test.shape)
print('테스트 target 데이터의 크기 :',decoder_input_test.shape)
print('테스트 target 레이블의 크기 :',decoder_target_test.shape)

훈련 source 데이터의 크기 : (29700, 7)
훈련 target 데이터의 크기 : (29700, 16)
훈련 target 레이블의 크기 : (29700, 16)
테스트 source 데이터의 크기 : (3300, 7)
테스트 target 데이터의 크기 : (3300, 16)
테스트 target 레이블의 크기 : (3300, 16)


In [24]:
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, Masking
from tensorflow.keras.models import Model

In [25]:
#Masking은 패딩 토큰인 숫자 0의 경우 연산을 제외
embedding_dim=64
hidden_units=64

In [26]:
#인코더
encoder_inputs=Input(shape=(None,))
enc_emb=Embedding(src_vocab_size, embedding_dim)(encoder_inputs)
enc_masking=Masking(mask_value=0.0)(enc_emb)
encoder_lstm=LSTM(hidden_units, return_state=True)
encoder_outputs, state_h, state_c=encoder_lstm(enc_masking)
encoder_states=[state_h, state_c]

In [31]:
#디코더
decoder_inputs=Input(shape=(None,))
dec_emb_layer=Embedding(tar_vocab_size, embedding_dim)#임베딩 층
dec_emb=dec_emb_layer(decoder_inputs)# 밑에서 재사용하기 위해 분리시킴
dec_masking=Masking(mask_value=0.0)(dec_emb)
decoder_lstm=LSTM(hidden_units, return_sequences=True, return_state=True)
decoder_outputs, _, _=decoder_lstm(dec_masking, initial_state=encoder_states)
decoder_dense=Dense(tar_vocab_size, activation='softmax')
decoder_outputs=decoder_dense(decoder_outputs)#밑에서 재사용하기 위해 분리시킴

In [32]:
with tf.device('/device:GPU:0'):
  model=Model([encoder_inputs, decoder_inputs], decoder_outputs)
  model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['acc'])
  model.fit(x=[encoder_input_train, decoder_input_train], y=decoder_target_train, validation_data=([encoder_input_test, decoder_input_test], decoder_target_test), batch_size=128, epochs=50)

Epoch 1/50
233/233 [==============================] - 31s 92ms/step - loss: 3.2858 - acc: 0.6150 - val_loss: 1.8770 - val_acc: 0.6434
Epoch 2/50
233/233 [==============================] - 9s 39ms/step - loss: 1.6966 - acc: 0.7275 - val_loss: 1.5260 - val_acc: 0.7720
Epoch 3/50
233/233 [==============================] - 9s 38ms/step - loss: 1.4117 - acc: 0.7862 - val_loss: 1.3291 - val_acc: 0.7983
Epoch 4/50
233/233 [==============================] - 7s 31ms/step - loss: 1.2565 - acc: 0.8049 - val_loss: 1.2238 - val_acc: 0.8109
Epoch 5/50
233/233 [==============================] - 8s 35ms/step - loss: 1.1561 - acc: 0.8149 - val_loss: 1.1506 - val_acc: 0.8176
Epoch 6/50
233/233 [==============================] - 8s 33ms/step - loss: 1.0758 - acc: 0.8234 - val_loss: 1.0898 - val_acc: 0.8252
Epoch 7/50
233/233 [==============================] - 7s 31ms/step - loss: 1.0051 - acc: 0.8321 - val_loss: 1.0379 - val_acc: 0.8322
Epoch 8/50
233/233 [==============================] - 8s 33ms/step -

In [34]:
#decoder의 테스트과정은 교사강요의 훈련과정과 동작 방식이 다르므로 모델을 다시 설계해야 한다.
#encoder
encoder_model=Model(encoder_inputs, encoder_states)

#디코더 설계 시작
decoder_state_input_h=Input(shape=(hidden_units,))
decoder_state_input_c=Input(shape=(hidden_units,))
decoder_state_inputs=[decoder_state_input_h, decoder_state_input_c]

dec_emb2=dec_emb_layer(decoder_inputs)
decoder_outputs2, state_h2, state_c2=decoder_lstm(dec_emb2, initial_state=decoder_state_inputs)
decoder_states2=[state_h2, state_c2]

#모든 시점에 대하여 단어 예측
decoder_outputs2=decoder_dense(decoder_outputs2)

#수정된 디코더
decoder_model=Model([decoder_inputs]+decoder_state_inputs, [decoder_outputs2]+decoder_states2)

In [41]:
def decode_sequence(input_seq):
  #입력으로부터 인코더의 마지막 시점의 은닉상태, 셀상태를 얻는다.
  states_value=encoder_model.predict(input_seq)

  #<sos>에 해당하는 정수 생성
  target_seq=np.zeros((1,1))
  target_seq[0,0]=tar_to_index['<sos>']

  stop_condition=False
  decoded_sentence=""

  #stop_condition=True까지 무한반복
  while not stop_condition:
    #states_value를 현 시점의 초기상태로 사용
    output_tokens, h, c=decoder_model.predict([target_seq]+states_value)

    #예측 결과를 단어로 변환
    sampled_token_index=np.argmax(output_tokens[0,-1,:]) #softmax함수로 가장 큰 값을 배정
    sampled_char=index_to_tar[sampled_token_index]

    #현재 시점의 예측 단어를 예측 문장에 추가
    decoded_sentence+=' '+sampled_char

    #중단시점
    if(sampled_char=='<eos>' or len(decoded_sentence)>50):
      stop_condition=True

    #현재 시점의 예측 결과를 다음 시점의 입력으로 사용
    target_seq=np.zeros((1,1))
    target_seq[0,0]=sampled_token_index #현재 시점의 예측 결과

    states_value=[h,c]

  return decoded_sentence

In [42]:
def seq_to_src(input_seq):
  sentence=''
  for encoded_word in input_seq:
    if(encoded_word!=0):
      sentence+=index_to_src[encoded_word]+' '
  return sentence

def seq_to_tar(input_seq):
  sentence=''
  for encoded_word in input_seq:
    if(encoded_word!=0 and encoded_word!=tar_to_index['<sos>']and encoded_word!=tar_to_index['<eos>']):
      sentence+=index_to_tar[encoded_word]+' '
  return sentence

In [45]:
for seq_index in [1, 70, 100, 300, 1001]:
  input_seq = encoder_input_train[seq_index: seq_index + 1]
  decoded_sentence = decode_sequence(input_seq)

  print("입력문장 :",seq_to_src(encoder_input_train[seq_index]))
  print("정답문장 :",seq_to_tar(decoder_input_train[seq_index]))
  print("번역문장 :",decoded_sentence[1:-5])
  print("-"*50)

1/1 [==============================] - 0s 21ms/step
입력문장 : are they in love ? 
정답문장 : sont ils amoureux ? 
번역문장 : sont ils amoureux ? 
--------------------------------------------------
1/1 [==============================] - 0s 24ms/step
입력문장 : he sang a song . 
정답문장 : il a chante une chanson . 
번역문장 : il a chante une chanson . 
--------------------------------------------------
1/1 [==============================] - 0s 23ms/step
입력문장 : i felt very happy . 
정답문장 : je me sentais tres heureux . 
번역문장 : je me suis sentie tres heureux . 
--------------------------------------------------
1/1 [==============================] - 0s 23ms/step
입력문장 : do you have a copy ? 
정답문장 : en avez vous une copie ? 
번역문장 : en as tu un exemplaire ? 
--------------------------------------------------
1/1 [==============================] - 0s 23ms/step
입력문장 : i can protect you . 
정답문장 : je peux te proteger . 
번역문장 : je peux te proteger . 
--------------------------------------------------
